In [ ]:
from optimize_model import optimize_model
from cobra_utils import (
    load_model,
    set_default_bounds,
    fetch_norm_sample_mbx_data,
    fetch_mbx_constr_list,
    slack_constraints,
)

# Define paths
json_model_path = "example_data/models/microbiota_model_diet_Case_1_18_month.json"
matlab_model_path = "example_data/models/microbiota_model_diet_Case_1_18_month.mat"
mbx_path = "example_data/metabolomics_data.csv"
avg_eu_diet_path = "example_data/AverageEU_diet_fluxes.txt"

# Load the model
model = load_model(json_model_path)

# Set the solver interface
model.solver = "gurobi"

# Get the model-specific metabolomics data
mbx_metab_norm_dict = fetch_norm_sample_mbx_data(
    model_input=model,
    mbx_filepath=mbx_path,
    match_key_output_filepath="example_outputs",
)

# Set all fecal exchange reaction lower bounds to zero
set_default_bounds(model)

# Fetch and test the constraint list
mbx_constraints = fetch_mbx_constr_list(model, mbx_metab_norm_dict)

In [ ]:
# Fetch the slack constraints if possible
# FIXME needs to add a check for the pos and neg slack constraints and implement optimization with them
slack_constr = slack_constraints(model, mbx_constraints)

In [ ]:
model.problem.objective = 0
solution = model.optimize()
for name in solution.fluxes.index:
    if name.endswith("slack"):
        print(f"{name}\t{solution.fluxes[name]}")

In [ ]:
existing_vars = []
for var in model.variables:
    if (
        var.name.endswith("_slack_pos")
        or var.name.endswith("_slack_neg")
        or var.name.endswith("_slack")
    ):
        model.remove_cons_vars(var)
        existing_vars.append(var.name)

print(existing_vars)

In [ ]:
# model.objective = 0
# solution = model.optimize()
# print(solution.fluxes)

In [ ]:
# for rxn_id in [rxn.id for rxn in model.reactions if rxn.id.startswith("EX_") and rxn.id.endswith("[fe]")]:
#     model.objective = model.reactions.get_by_id(rxn_id)
#     solution = model.optimize()
#     print(f"{model.reactions.get_by_id(rxn_id).id}:\t{solution.objective_value}")